<h1><center>Laboratorio N° 03: Pivoting y Grouping Sets

<h2>Parte 01. Escribir consultas que utilizan PIVOT.

1.1. Abrir la base de datos TSQL.

In [2]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00.001

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

In [8]:
CREATE VIEW Production.VistaCategoriasPorAño AS
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    c.CategoryName, YEAR(o.OrderDate);

Commands completed successfully.

Total execution time: 00:00:00.004

1.3. Probar la vista creada.

In [9]:
SELECT * FROM Production.VistaCategoriasPorAño

(24 rows affected)

Total execution time: 00:00:00.081

CategoryName,Año,TotalCantidad
Meat/Poultry,2006,950
Seafood,2008,2716
Dairy Products,2007,4374
Grains/Cereals,2008,1377
Confections,2007,4137
Produce,2008,858
Beverages,2006,1842
Condiments,2008,1441
Dairy Products,2008,2689
Seafood,2007,3679


1.4. Utilizar PIVOT entre categorías y años de ordenes.

In [10]:
SELECT 
    CategoryName, 
    [2018], 
    [2019], 
    [2020], 
    [2021], 
    [2022]
FROM 
    (
        SELECT 
            CategoryName,
            Año,
            TotalCantidad
        FROM 
            Production.VistaCategoriasPorAño
    ) AS SourceTable
PIVOT 
    (
        SUM(TotalCantidad)
        FOR Año IN ([2018], [2019], [2020], [2021], [2022])
    ) AS PivotTable;

(8 rows affected)

Total execution time: 00:00:00.046

CategoryName,2018,2019,2020,2021,2022
Beverages,NULL,NULL,NULL,NULL,NULL
Condiments,NULL,NULL,NULL,NULL,NULL
Confections,NULL,NULL,NULL,NULL,NULL
Dairy Products,NULL,NULL,NULL,NULL,NULL
Grains/Cereals,NULL,NULL,NULL,NULL,NULL
Meat/Poultry,NULL,NULL,NULL,NULL,NULL
Produce,NULL,NULL,NULL,NULL,NULL
Seafood,NULL,NULL,NULL,NULL,NULL


1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

In [11]:
SELECT 
    CategoryName, 
    [2018], 
    [2019], 
    [2020], 
    [2021], 
    [2022]
INTO 
    dbo.ResultadoPivot
FROM 
    (
        SELECT 
            CategoryName,
            Año,
            TotalCantidad
        FROM 
            Production.VistaCategoriasPorAño
    ) AS SourceTable
PIVOT 
    (
        SUM(TotalCantidad)
        FOR Año IN ([2018], [2019], [2020], [2021], [2022])
    ) AS PivotTable;

(8 rows affected)

Total execution time: 00:00:00.044

1.6. Probrar la tabla generada

In [12]:
SELECT * 
FROM dbo.ResultadoPivot;

(8 rows affected)

Total execution time: 00:00:00.016

CategoryName,2018,2019,2020,2021,2022
Beverages,NULL,NULL,NULL,NULL,NULL
Condiments,NULL,NULL,NULL,NULL,NULL
Confections,NULL,NULL,NULL,NULL,NULL
Dairy Products,NULL,NULL,NULL,NULL,NULL
Grains/Cereals,NULL,NULL,NULL,NULL,NULL
Meat/Poultry,NULL,NULL,NULL,NULL,NULL
Produce,NULL,NULL,NULL,NULL,NULL
Seafood,NULL,NULL,NULL,NULL,NULL


1.7. Utilizar UNPIVOT.

In [13]:
SELECT 
    CategoryName, 
    Año, 
    TotalCantidad
FROM 
    (
        SELECT 
            CategoryName, 
            [2018], 
            [2019], 
            [2020], 
            [2021], 
            [2022]
        FROM 
            dbo.ResultadoPivot
    ) AS p
UNPIVOT 
    (
        TotalCantidad FOR Año IN ([2018], [2019], [2020], [2021], [2022])
    ) AS unpvt;

(0 rows affected)

Total execution time: 00:00:00.021

CategoryName,Año,TotalCantidad


1.8. Limpiar los cambios realizados.


In [14]:
IF OBJECT_ID('dbo.ResultadoPivot', 'U') IS NOT NULL 
    DROP TABLE dbo.ResultadoPivot;

IF OBJECT_ID('Production.VistaCategoriasPorAño', 'V') IS NOT NULL 
    DROP VIEW Production.VistaCategoriasPorAño;

Commands completed successfully.

Total execution time: 00:00:00.011

<h2>Parte 02. Escribir consultas que utilizan UNPIVOT.

2.1. Ejecutar la siguiente vistas

2.2. Elaborar una consulta sin utilizar GROUPING SETS.

In [20]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    c.CategoryName, YEAR(o.OrderDate);

(24 rows affected)

Total execution time: 00:00:00.018

CategoryName,Año,TotalCantidad
Meat/Poultry,2006,950
Seafood,2008,2716
Dairy Products,2007,4374
Grains/Cereals,2008,1377
Confections,2007,4137
Produce,2008,858
Beverages,2006,1842
Condiments,2008,1441
Dairy Products,2008,2689
Seafood,2007,3679


2.3. Consultar con GROUPING SETS.

In [21]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    GROUPING SETS (
        (c.CategoryName, YEAR(o.OrderDate)),  -- Total por categoría y año
        (c.CategoryName),                      -- Total por categoría
        (YEAR(o.OrderDate)),                   -- Total por año
        ()                                     -- Total general
    );

(36 rows affected)

Total execution time: 00:00:00.039

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Condiments,2006,962
Confections,2006,1357
Dairy Products,2006,2086
Grains/Cereals,2006,549
Meat/Poultry,2006,950
Produce,2006,549
Seafood,2006,1286
NULL,2006,9581
Beverages,2007,3996


2.4. Consultar con CUBE.

In [17]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    CUBE (c.CategoryName, YEAR(o.OrderDate));

(36 rows affected)

Total execution time: 00:00:00.119

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Condiments,2006,962
Confections,2006,1357
Dairy Products,2006,2086
Grains/Cereals,2006,549
Meat/Poultry,2006,950
Produce,2006,549
Seafood,2006,1286
NULL,2006,9581
Beverages,2007,3996


2.5. Consultar con ROLLUP

In [18]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    ROLLUP (c.CategoryName, YEAR(o.OrderDate));

(33 rows affected)

Total execution time: 00:00:00.035

CategoryName,Año,TotalCantidad
Beverages,2006,1842
Beverages,2007,3996
Beverages,2008,3694
Beverages,NULL,9532
Condiments,2006,962
Condiments,2007,2895
Condiments,2008,1441
Condiments,NULL,5298
Confections,2006,1357
Confections,2007,4137


2.6. Utilizar Grouping_ID.

In [19]:
SELECT 
    c.CategoryName,
    YEAR(o.OrderDate) AS Año,
    SUM(od.qty) AS TotalCantidad,
    GROUPING_ID(c.CategoryName, YEAR(o.OrderDate)) AS GrpID
FROM 
    Production.Categories c
INNER JOIN 
    Production.Products p ON c.CategoryID = p.CategoryID
INNER JOIN 
    Sales.OrderDetails od ON p.ProductID = od.ProductID
INNER JOIN 
    Sales.Orders o ON od.OrderID = o.OrderID
GROUP BY 
    ROLLUP (c.CategoryName, YEAR(o.OrderDate));

(33 rows affected)

Total execution time: 00:00:00.064

CategoryName,Año,TotalCantidad,GrpID
Beverages,2006,1842,0
Beverages,2007,3996,0
Beverages,2008,3694,0
Beverages,NULL,9532,1
Condiments,2006,962,0
Condiments,2007,2895,0
Condiments,2008,1441,0
Condiments,NULL,5298,1
Confections,2006,1357,0
Confections,2007,4137,0


2.7. Finalmente limpiar los cambios

In [22]:
-- Eliminar la tabla ResultadoPivot si existe
IF OBJECT_ID('dbo.ResultadoPivot', 'U') IS NOT NULL 
    DROP TABLE dbo.ResultadoPivot;

-- Eliminar la vista VistaCategoriasPorAño si existe
IF OBJECT_ID('Production.VistaCategoriasPorAño', 'V') IS NOT NULL 
    DROP VIEW Production.VistaCategoriasPorAño;

Commands completed successfully.

Total execution time: 00:00:00.001

<h2>Ejercicio 03: Escribir consultas que usan las clausulas GROUPING SETS, CUBE y ROLLUP

3.1. Escribir una sentencia SELECT contra la tabla Sales.Customers y traer las columnas country, city
y una columna calculada noofcustomers como una cantidad de clientes. Traer multiples grouping
sets basados en las columnas country y city, la columna country, la columna city, y una columna con un grouping set vacio.

In [23]:
SELECT 
    country,
    city,
    COUNT(*) AS noofcustomers
FROM 
    Sales.Customers
GROUP BY 
    GROUPING SETS (
        (country, city),   -- Total por país y ciudad
        (country),         -- Total por país
        (city),            -- Total por ciudad
        ()                 -- Total general
    );

(160 rows affected)

Total execution time: 00:00:00.043

country,city,noofcustomers
Germany,Aachen,1
NULL,Aachen,1
USA,Albuquerque,1
NULL,Albuquerque,1
USA,Anchorage,1
NULL,Anchorage,1
Denmark,Århus,1
NULL,Århus,1
Spain,Barcelona,1
NULL,Barcelona,1


3.2. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Día de la columna orderdate como orderday.
<li>Valor de Total de Ventas utilizando la columna val como salesvalue.
<li>Devolver todas los posibles  grouping sets basados en las columnas orderyear, ordermonth, y orderday.

In [24]:
SELECT 
    YEAR(orderdate) AS orderyear,
    MONTH(orderdate) AS ordermonth,
    DAY(orderdate) AS orderday,
    SUM(val) AS salesvalue
FROM 
    Sales.OrderValues
GROUP BY 
    GROUPING SETS (
        (YEAR(orderdate), MONTH(orderdate), DAY(orderdate)),  -- Total por año, mes y día
        (YEAR(orderdate), MONTH(orderdate)),                   -- Total por año y mes
        (YEAR(orderdate), DAY(orderdate)),                     -- Total por año y día
        (MONTH(orderdate), DAY(orderdate)),                    -- Total por mes y día
        (YEAR(orderdate)),                                     -- Total por año
        (MONTH(orderdate)),                                    -- Total por mes
        (DAY(orderdate)),                                      -- Total por día
        ()                                                     -- Total general
    );

(948 rows affected)

Total execution time: 00:00:00.065

orderyear,ordermonth,orderday,salesvalue
2007,1,1,6931.60
2008,1,1,1738.00
NULL,1,1,8669.60
2007,4,1,851.20
2008,4,1,11549.89
NULL,4,1,12401.09
2007,5,1,5636.96
2008,5,1,5448.57
NULL,5,1,11085.53
2007,7,1,142.50


3.3. Copiar la consulta previa y modificarla para utilizar la clausula ROLLUP en vez de CUBE.

In [25]:
SELECT 
    YEAR(orderdate) AS orderyear,
    MONTH(orderdate) AS ordermonth,
    DAY(orderdate) AS orderday,
    SUM(val) AS salesvalue
FROM 
    Sales.OrderValues
GROUP BY 
    ROLLUP (
        YEAR(orderdate), 
        MONTH(orderdate), 
        DAY(orderdate)
    );

(507 rows affected)

Total execution time: 00:00:00.029

orderyear,ordermonth,orderday,salesvalue
2006,7,4,440.00
2006,7,5,1863.40
2006,7,8,2206.66
2006,7,9,3597.90
2006,7,10,1444.80
2006,7,11,556.62
2006,7,12,2490.50
2006,7,15,517.80
2006,7,16,1119.90
2006,7,17,1614.88


3.4. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Columna Calculada con el alias groupid (usar la función GROUPING_ID con los campos order year y order month como parametros).
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Valor Total de Ventas utilizando la columna val como salesvalue.
<li>Desde Año y mes month forman una jerarquia, retornar todos los grouping sets interesantes basados en las columnas orderyear y ordermonth y ordenar el resultado por groupid, orderyear, and ordermonth